# Map Making [Compact Version]

In this lesson we cover the mapmaking problem and current and available TOAST mapmaking facilities
* `OpMadam` -- interface to `libMadam`, a parallel Fortran library for destriping and mapping signal
* `OpMapmaker` -- nascent implementation of a native TOAST mapmaker with planned support for a host of systematics templates

In [ ]:
# Load tools
import sys
sys.path.insert(0, "..")
from lesson_tools import (
    fake_focalplane
)

import toast
import toast.todmap
import toast.pipeline_tools
from toast.mpi import MPI

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import matplotlib as mpl
import re

import todgb

mpiworld, procs, rank = toast.mpi.get_world()
comm = toast.mpi.Comm(mpiworld)
# A pipeline would create the args object with argparse

# Capture C++ output in the jupyter cells
%reload_ext wurlitzer

# Everything Else

In [ ]:
# Recall information on the GB focal plane. (written by Seongsu Kim)
Zp = np.genfromtxt('../04_Simulated_Instrument_Signal/Zp.txt', delimiter = '\t')
fp = np.genfromtxt('../04_Simulated_Instrument_Signal/fp.txt', delimiter = '\t')
angle_array = np.genfromtxt('../04_Simulated_Instrument_Signal/angle.txt', delimiter = '\t')

# If you want to select a smaller number of detectors to run the simulation
# more quickly, you can use any of the following:

#Input detector information for selected few detectors
# [number, Zpx, Zpy, Zpz] pointing in 3d space 
# [number, fpx, fpy] position on focal plane
# [number, fp angle, sky angle] polarization angles
ListZp1 = [1,-0.0272211,0.0311863,0.999143] # Detectors nos. 5 and 6
ListZp2 = [6,-0.0135674,0.023398,0.999634]
Listfp1 = [1,-15.6,13.51] 
Listfp2 = [6,11.7,6.755]
Listangle1 = [1,-22.5,-22.8042]
Listangle2 = [6,22.5,22.7159]
#Zp_ = np.array([np.array(ListZp1),np.array(ListZp2)])
#fp_ = np.array([np.array(Listfp1),np.array(Listfp2)])
#angle_array_ = np.array([np.array(Listangle1),np.array(Listangle2)])

# Or you could construct a text file with relevant info and simply call them.
Zp_ = np.genfromtxt('../04_Simulated_Instrument_Signal/Zpalt.txt', delimiter = '\t')
fp_ = np.genfromtxt('../04_Simulated_Instrument_Signal/fpalt.txt', delimiter = '\t')
angle_array_ = np.genfromtxt('../04_Simulated_Instrument_Signal/anglealt.txt', delimiter = '\t')

GBF = [Zp,fp,angle_array]
Sample = [Zp_,fp_,angle_array_]

Tw = "../04_Simulated_Instrument_Signal/weather_Tenerife.fits"
Aw = "../04_Simulated_Instrument_Signal/weather_Atacama.fits"

#####FocalPlane, Weather
wh = GBF  # Which focal plane layout?
wf = Tw  # Weather File

### GB Scan Parameters

## Functions

In [ ]:
def readtime(scandur):
    sday = (re.findall(r"(\d+) Day", scandur))
    shour = (re.findall(r"(\d+) Hour", scandur))
    sminute = (re.findall(r"(\d+) Minute", scandur))
    ssecond = (re.findall(r"(\d+) Seconds", scandur))

    slist = [sday,shour,sminute,ssecond]
    nlist = [0,0,0,0]
    for i in range(0,3):
        if len(slist[i])== 0:
            nlist[i] = 0
        else:
            nlist[i] = int((slist[i])[0])

    nday,nhour,nminute,nsecond = nlist
    t_ = nday*24*60*60 + nhour*60*60 + nminute*60 + nsecond # Scan duration in the unit of seconds
    return t_

In [ ]:
def FocPlane(w,FK,NET):
    Zp,fp,angle_array = w[0],w[1],w[2]
    # set unit vector
    z_vec = np.array([0,0,1])
    y_vec = np.array([0,1,0])
    data_len = len(Zp.T[0])

    Zp_vector = (Zp.T[1:]).T
    for i in range(data_len):
        Zp_vector[i] = Zp_vector[i]/np.sqrt(np.dot(Zp_vector[i],Zp_vector[i]))
        
    # angle theta
    # data_len = 2
    theta = np.zeros(data_len)
    theta_print = np.zeros(data_len)
    for i in range(data_len):
        theta[i] = np.dot(z_vec,Zp_vector[i])
        theta_print[i] = np.rad2deg(np.arccos(theta[i]))
        theta[i] = np.arccos(theta[i])
        theta_print[i] = round(theta_print[i],2)
        
    # build focalplane in length scale
    diff_angle = angle_array.T[2]-angle_array.T[1]
    color = diff_angle + max(diff_angle)
    x_fp = fp.T[1]
    y_fp = fp.T[2]
    c_fp = fp.T[0]
    
    # angle phi
    # project Zp onto xy plane (Zp_z = 0)
    Zp_proj = np.zeros((data_len,3))
    for i in range(data_len):
        Zp_proj[i][0] = Zp_vector[i][0]
        Zp_proj[i][1] = Zp_vector[i][1]
    # normalize
    for i in range(data_len):
        Zp_proj[i] = Zp_proj[i]/np.sqrt(np.dot(Zp_proj[i],Zp_proj[i]))
        
    phi = np.zeros(data_len)
    phi_print = np.zeros(data_len)
    for i in range(data_len):
        phi[i] = np.dot(y_vec,Zp_proj[i])
        if Zp_proj[i][0] < 0:
            phi[i] = np.arccos(phi[i])
        else:
            phi[i] = -np.arccos(phi[i])
        phi_print[i] = round(np.rad2deg(phi[i]),2)
    
    # build quaternion
    pa = np.pi/8
    # pa = 0

    # find index
    if data_len>10:
        left = [6,7,8,9,15,16,17,18,29,30,31,32,41,40,39,38,55,54,53,52,64,63,62,61,78,77,76,75,
                87,86,85,84,101,100,99,98,110,109,108,107,124,123,122,121,133,132,131,130,147,146,
                145,144,156,155,154,153]
        all = np.linspace(0,161,162)
        right = []
        for i in range(data_len):
            if i not in left:
                right.append(i)
    else:
        left, right =[],[]
        for i in range(data_len):
            #print("angle_array")
            #print(angle_array[i])
            if angle_array[i][1]<0:
                right.append(i),
            else: left.append(i)
    
    import toast.qarray as qa
    quat = np.zeros((data_len,4))
    for i in range(data_len):
        if i in right:
            quat[i] = qa.from_angles(theta[i],phi[i],-np.pi/2-phi[i]-pa)
        else:
            quat[i] = qa.from_angles(theta[i],phi[i],-np.pi/2-phi[i]+pa)
    
    # build telescope dict
    tel_dict = {}
    detname = np.linspace(0,data_len-1,data_len)
    for i in range(data_len):
        detname[i] = int(detname[i])
    
    for i in range(data_len):
        name = 'det_{}'.format(i) # det_(index)
        tel_dict[name] = {
            'quat': quat[int(i)],
            'epsilon': 0,
            'rate': 1000,
            'alpha': 1,
            'NET': NET,
            'fmin': 0,
            'fknee': FKnee,
            'fwhm_arcmin': 30,
        }
        
    %matplotlib inline 
    ind = 1
    key = 'det_{}'.format(ind)

    # list of dict of detectors, sort(), 순서대로
    detnames = list(tel_dict.keys()) 

    # assign "qaut"(quaternion) value of fp[i] in i_th x
    detquat = {x: tel_dict[x]["quat"] for x in detnames} 
    detquat_ind = detquat[key]
    detquat_plot = {'{}'.format(key): detquat_ind}
    # print(detquat_plot)

    # assign "fwhm_arcmin" value of fp[i] in i_th x, 
    # fwhm, full width at half maximum
    detfwhm = {x: tel_dict[x]["fwhm_arcmin"] for x in detnames}
    detfwhm_ind = detfwhm[key]
    detfwhm_plot = {'{}'.format(key): detfwhm_ind}

    # labels
    detlabels = {x: x for x in detnames}
    detlabels_ind = detlabels[key]
    detlabels_plot = {'{}'.format(key): detlabels_ind}

    # ploariztion colors
    detpolcol = {x: "red" for i, x in enumerate(detnames)}
    
    #plot
    toast.tod.plot_focalplane(
        detquat, 20.0, 20.0, None,polcolor = detpolcol, fwhm=detfwhm, labels=detlabels 
    )
    
    global det_length
    det_length = len(tel_dict)
    return tel_dict

In [ ]:
def MakeInputData(tel_dict,weatherfile,sr,t_,SCR):
    
    samplerate = sr
    time = t_
    # seconds
    sampleno = samplerate*time
    SCANrate = SCR
    # degrees per second
    # GB: 120
    
    #some MPI stuff
    mpiworld, procs, rank = toast.mpi.get_world()
    comm = toast.mpi.Comm(mpiworld)
    
    # Setup은 위에서 정의하고 setup 그대로 넣어주면 됨
    ! toast_ground_schedule.py \
        --site-lat "28.4746" \
        --site-lon "-16.3081" \
        --site-alt 2390 \
        --site-name Teide \
        --telescope LAT \
        --start "2020-03-20 10:30:00" \
        --stop "2020-03-20 11:30:00" \
        --patch-coord C \
        --patch small_patch,1,30,28,10 \
        --ces-max-time 86400 \
        --out schedule.txt

    ! cat schedule.txt
    
    class args:
        split_schedule = None
        schedule = "schedule.txt"
        sort_schedule = False  # Matters for parallelization
        weather = weatherfile #THIS IS WHERE THE WEATHER GOES
        #so the code won't run without it. We will leave it like this for the time being.
        sample_rate = 113  # Hz
        scan_rate = 120  # deg / s
        # Use an artifially low acceleration to show the turn-arounds better
        scan_accel = 0  # deg / s^2
        hwp_rpm = None
        hwp_step_deg = None
        hwp_step_time_s = None
        fov = 3.0  # Field-of-view in degrees
    
    focalplane = toast.pipeline_tools.Focalplane(
        tel_dict, sample_rate=args.sample_rate
    )
    
    # Load the observing schedule, append weather and focalplane to it 
    schedules = toast.pipeline_tools.load_schedule(args, comm)
    toast.pipeline_tools.load_weather(args, comm, schedules)
    # There could be more than one observing schedule, but not this time
    schedule = schedules[0]
    schedule.telescope.focalplane = focalplane
    ces = schedule.ceslist[0]  # normally we would loop over entries
    totsamples = int((ces.stop_time - ces.start_time) * args.sample_rate)
    telescope = schedule.telescope  # shorthand
    
    #organize processes
    if comm.comm_group is not None:
        # Available detectors should be split between processes in the group
        ndetrank = comm.comm_group.size
    else:
        ndetrank = 1
    
    tod_ = toast.todmap.TODGround(
        comm.comm_group,
        telescope.focalplane.detquats,
        totsamples,
        detranks=ndetrank,
        firsttime=ces.start_time,
        rate=args.sample_rate,
        site_lon=telescope.site.lon,
        site_lat=telescope.site.lat,
        site_alt=telescope.site.alt,
        azmin=ces.azmin,
        azmax=ces.azmax,
        el=ces.el,
        scanrate=args.scan_rate,
        scan_accel=args.scan_accel,
        
        hwprpm=args.hwp_rpm,
        hwpstep=args.hwp_step_deg,
        hwpsteptime=args.hwp_step_time_s,
    )
    
    tod = todgb.TODGb(
        detectors=tel_dict,
        samples=sampleno, ############# 3 Variables out of which we input 2 and leave one blank (or fit it in redundantly)
        mpicomm=comm.comm_group,
        boresight_angle=0,
        firsttime=0.0,
        rate=samplerate, # May have to do with scanning time and stuff
        # ############# 3 Variables out of which we input 2 and leave one blank (or fit it in redundantly)
        site_lon=telescope.site.lon,
        site_lat=telescope.site.lat,
        site_alt=telescope.site.alt,
        el=60,
        scanrate=SCANrate, # May have to do with scanning time and stuff
        scan_accel=0.1, # May have to do with scanning time and stuff
        CES_start=None, ############# 3 Variables out of which we input 2 and leave one blank (or fit it in redundantly)
        CES_stop=None, ############# 3 Variables out of which we input 2 and leave one blank (or fit it in redundantly)
        sun_angle_min=90,
        sampsizes=None,
        sampbreaks=None,
        coord="C",
        report_timing=True,
        hwprpm=args.hwp_rpm,
        hwpstep=args.hwp_step_deg,
        hwpsteptime=args.hwp_step_time_s,
        sinc_modulation=False,
    )
    
    return [tod,telescope,tod_,ces]


In [ ]:
def DefineOBS_data(m):
    tod,telescope,tod_,ces = m[0],m[1],m[2],m[3]
    
    #some MPI stuff
    mpiworld, procs, rank = toast.mpi.get_world()
    comm = toast.mpi.Comm(mpiworld)
    
    obs = {}
    obs["name"] = "CES-{}-{}-{}-{}-{}".format(
        telescope.site.name, telescope.name, ces.name, ces.scan, ces.subscan
    )
    obs["tod"] = tod
    obs["baselines"] = None
    obs["noise"] = telescope.focalplane.noise
    obs["id"] = int(ces.mjdstart * 10000)
    obs["intervals"] = None # tod_.subscans
    obs["site"] = telescope.site
    obs["site_name"] = telescope.site.name
    obs["site_id"] = telescope.site.id
    obs["altitude"] = telescope.site.alt
    obs["weather"] = telescope.site.weather
    obs["telescope"] = telescope
    obs["telescope_name"] = telescope.name
    obs["telescope_id"] = telescope.id
    obs["focalplane"] = telescope.focalplane.detector_data
    obs["fpradius"] = telescope.focalplane.radius
    obs["start_time"] = ces.start_time
    obs["season"] = ces.season
    obs["date"] = ces.start_date
    obs["MJD"] = ces.mjdstart
    obs["rising"] = ces.rising
    obs["mindist_sun"] = ces.mindist_sun
    obs["mindist_moon"] = ces.mindist_moon
    obs["el_sun"] = ces.el_sun
    
    data = toast.Data(comm)
    data.obs.append(obs)
    
    ### Degubbing
    ### I mean Debugging
    
    '''
    print('################################')
    #print(data.obs)
    print(len(data.obs))
    
    print('data.obs type : '+str(type(data.obs)))
    if type(data.obs)==list:
        if type(data.obs[0])==dict:
            print('data.obs keys : '+str(dict.keys(data.obs[0])))  
    print('data type='+str(type(data)))
    if type(data)==list:
        if type(data[0])==dict:
            print('data keys : '+str(dict.keys(data[0])))  
    
    '''
        
    return [data,tod,obs]
# 'data' is of <class 'toast.dist.Data'>
# and 'data.obs' is a list whose items are <class 'dict'>

In [ ]:
def mapmachen(scan,res,signalparams,Gain,datatuple):
    s=signalparams
    sr,scandur,scr,sc_ac,SetBaseline = scan[0],scan[1],scan[2],scan[3],scan[4]
    nside_inputmap,nside_mapmaking,nnz_manual = res[0],res[1],res[2]
    Whichmap,SetHubble,SetUnit,SetLmax,InputType,InputNside,SetFWHM = s[0],s[1],s[2],s[3],s[4],s[5],s[6]
    data,tod,obs = datatuple[0],datatuple[1],datatuple[2]
    
    # Define arguments to execute mapmaker
    class args:
        sample_rate = sr  # Hz
        scan_rate = scr  # deg / s
        scan_accel = sc_ac  # deg / s^2
        hwp_rpm = None
        hwp_step_deg = None
        hwp_step_time_s = None
        #spin_period_min = 0.05 
        #spin_angle_deg = 120*60
        #prec_period_min = 1000000 
        #prec_angle_deg = 0 
        coord = "E"
        nside = nside_mapmaking
        nnz = nnz_manual
        outdir = "maps"
        
    # Here we create an angular power spectrum that has the properties of the CMB.
    # CMB has a temperature fluctuation on the scale of 100 muK.

    import camb
    # given a cosmological parameter, this module gives you an angular power spectrum 
    # (to use as input CMB signal)
    pars = camb.CAMBparams()
    pars.set_cosmology(H0 = SetHubble)

    res = camb.get_results(pars)
    clss = res.get_total_cls(lmax=SetLmax,CMB_unit=SetUnit,raw_cl=True)
    Simulated_CMB = clss.T # output is L, [TT, EE, BB, TE]: and the required input for our signal is [TT, EE, BB, TE], L

    # Alternative Input Signal (can be adjusted to whatever scale you want)
    Lmax_Random = nside_mapmaking*2
    Simulated_Random = np.zeros([4, Lmax_Random + 1])
    Simulated_Random[0] = 1e0

    Simulated_Input = [Simulated_Random, Simulated_CMB]
    
    cls = Simulated_Input[InputType]
    sim_map = hp.synfast(cls, InputNside, lmax=SetLmax, fwhm=SetFWHM, new=True)
    ### Input map is written as a nest!!!
    hp.write_map("sim_map.fits", hp.reorder(sim_map, r2n=True), nest=True, overwrite=True)
    
    ####################
    # Compute Pointing
    name = "signal"
    toast.tod.OpCacheClear(name).exec(data)
    toast.todmap.OpPointingHpix(nside=args.nside, mode="IQU", nest=True).exec(data)
    
    # Clear cache
    toast.tod.OpCacheClear(name).exec(data)

    # Scan the signal from a map
    distmap = toast.map.DistPixels(
        data,
        nnz=nnz_manual,
        dtype=np.float32,
    )
    distmap.read_healpix_fits("sim_map_fixed.fits")

    toast.todmap.OpSimScan(input_map=distmap, out=name).exec(data)

    # Copy the sky signal and define it as 'sky_signal'
    toast.tod.OpCacheCopy(input=name, output="sky_signal", force=True).exec(data)
        
    # Simulate noise
    #1. Instrument Noise
    toast.tod.OpSimNoise(out=name, realization=0).exec(data)
    toast.tod.OpCacheCopy(input=name, output="without_atmos", force=True).exec(data)

    #2. Atmospheric Noise
    #'''
    atmsim = toast.todmap.OpSimAtmosphere(
        out=name,
        realization=0,  # Each MC will have a different realization
        zmax=1000,  # maximum altitude to integrate
        lmin_center=0.01,  # Dissipation scale
        lmin_sigma=0,
        lmax_center=10,  # Injection scale
        lmax_sigma=0,
        xstep=30, # Volume element size
        ystep=30,
        zstep=30,
        nelem_sim_max=10000,  # Target number of volume elements to consider at a time
        gain=Gain,  
        # If the wind is strong or the observation is long, the volume becomes
        # too large.  This parameter controls breaking the simulation into
        # disjoint segments
        wind_dist=10000,
        freq=100,  # Print progress to stdout and write out the correlation function
        )
    atmsim.exec(data)
    #'''
    
    toast.tod.OpCacheCopy(input=name, output="full_signal", force=True).exec(data)

    simdet = tod.cache['without_atmos_det_0']-tod.cache['sky_signal_det_0']
    simatm = tod.cache['full_signal_det_0']-tod.cache['without_atmos_det_0']
    simsig = tod.cache['sky_signal_det_0']
    
    # Copy the augmented signal and define it as 'full_signal'
    toast.tod.OpCacheCopy(input=name, output="full_signal", force=True).exec(data)
    # After this, recalling 'tod.cache[name(e.g. 'full_signal')]' gives the corresponding tod.
    # To see all keys: tod.cache.keys()
    
    #################### M A P M A K I N G ####################
    
    mapmaker = toast.todmap.OpMapMaker(
        nside=args.nside,
        nnz=nnz_manual, 
        name=name,
        outdir=args.outdir,
        outprefix="toast_test_",
        baseline_length=SetBaseline,
        ### Where we cut into intervals for removing noise offset
        # maskfile=self.maskfile_binary,
        # weightmapfile=self.maskfile_smooth,
        # subharmonic_order=None,
        iter_max=100,
        use_noise_prior=False,
        # precond_width=30,
    )
    mapmaker.exec(data)
    # This makes binned and destriped maps
    
    # See if the hitmap is written correctly
    hitmap_ = hp.read_map("maps/toast_test_hits.fits",nest=True)
    hitmap = hitmap = np.array(hitmap_, dtype='float')
    hitmap[hitmap == 0] = hp.UNSEEN
    global det_length
    a_ = int(np.sum(hitmap[hitmap != hp.UNSEEN]))
    b_ = sr*t_*det_length
    print('number of hits computed by hitmap: '+str(a_))
    print('number of hits computed manually: '+str(b_))
    if not a_ == b_:
        print('There is a problem with pointing calculations.')
    
    #################### P L O T T I N G ####################
    
    avgatm = np.mean(simatm-min(simatm))
    avgdet = np.mean(simdet-min(simdet))
    avgsig = np.mean(simsig-min(simsig))
    print('<Mean Width of Fluctuation (\u03BCK)>')
    print('Signal: '+str(avgsig))
    print('Detector Noise: '+str(avgdet))
    print('Atmospheric Noise: '+str(avgatm))
    
    ########## PRE-PLOT
    #setmin = min(simsig)-50    # Set thresholds for display on the map (accordingly, considering the scale of the signal)
    #setmax = max(simsig)+50
    binmap = hp.read_map("maps/toast_test_binned.fits")
    binmap[binmap == 0] = hp.UNSEEN    
    destriped = hp.read_map("maps/toast_test_destriped.fits")
    destriped[destriped == 0] = hp.UNSEEN    
    inmap = hp.read_map(Whichmap)
    inmap[hp.reorder(hitmap,n2r=True) == hp.UNSEEN] = hp.UNSEEN
    inmap_whole = hp.read_map(Whichmap)    
    
    if InputType == 0: 
        atype = 'Random Gaussian CLS'
        btype = ' of scale '+str(Simulated_Random[0])
    else: 
        atype = 'CMB CLS'
        btype = ' of Hubble Constant '+str(SetHubble)+' km/s/Mpc'
    l1 = 'Signal: '+atype+btype
    l2 = 'Detector-related factors:'
    l3 = 'NET: '+str(NET)+', FKnee: '+str(FKnee)
    l4 = 'Atmosphere-related factors:'
    l5 = 'Gain: '+str(Gain)
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('\n'.join(['Compare Signal/Noise Scales',l1,l2,l3,l4,l5]),y=1.16,ha = 'left')    
    fig.set_figheight(6)
    fig.set_figwidth(12)

    # 1 Detector noise PSD
    detfreq = datatuple[2]['noise'].freq('det_0')
    detpsd = datatuple[2]['noise'].psd('det_0')
    ax1.loglog(detfreq,detpsd)
    ax1.set_xlabel("Frequency [Hz]")
    ax1.set_ylabel("PSD [W/Hz]")
    ax1.set_title("Detector Noise PSD")

    # 2 Noise Scales
    times = tod.local_times()
    
    zads, listmax = ['a','d','s'], [max(simatm),max(simdet),max(simsig)]
    for i in [0,1,2]:
        if listmax[i] == max(listmax):
            zads[i] = 1
        elif listmax[i] == min(listmax):
            zads[i] = 3
        else: 
            zads[i] = 2

    ax2.plot(times, simatm, ".", label = 'Atmosphere', color='#80E0FF',zorder = zads[0])
    ax2.plot(times, simdet, ".", label = 'Detector', color='#B3FFB2',zorder = zads[1])
    ax2.plot(times, simsig, '.', label = 'sky', color='#DABEFF',zorder = zads[2])

    ax2.set_xlabel("Unix time [s]")
    ax2.set_ylabel("Atmosphere [\u03BCK]")
    ax2.set_title("Simulated Noise")
    ax2.legend()
    ax2.grid()

    plt.figure(figsize=[12, 8])
    plt.suptitle('\n'.join(['Simulated Maps',scandur+' Scan']),y=1.1)
    
    # 3 hitmap
    hp.mollview(hp.reorder(hitmap,n2r=True), sub=[3, 2, 1], nest=False, title="hits")

    # 4 binned map
    hp.mollview(binmap, sub=[3, 2, 2], nest=False, title="binned map", cmap="coolwarm",min=0.5*max(binmap),max=0.5*max(binmap))

    # 5 destriped
    hp.mollview(destriped, sub=[3, 2, 3], nest=False, title="destriped map", cmap="coolwarm",min=-0.5*max(destriped),max=0.5*max(destriped))

    # 6 input
    hp.mollview(inmap, sub=[3, 2, 4], nest=False, title="input map", cmap="coolwarm",min=-1.1*max(inmap),max=1.1*max(inmap))

    # 7 input whole
    hp.mollview(inmap_whole, sub=[2, 3, 5], nest=False, title="input map", cmap="coolwarm",min=1.1*min(inmap_whole),max=1.1*max(inmap_whole))
    
    # Plot the white noise covariance
    plt.figure(figsize=[12, 8])
    wcov = hp.read_map("maps/toast_test_npp.fits", None)
    wcov[:, wcov[0] == 0] = hp.UNSEEN
    hp.mollview(wcov[0], sub=[3, 3, 1], title="II", cmap="coolwarm")
    hp.mollview(wcov[1], sub=[3, 3, 2], title="IQ", cmap="coolwarm")
    hp.mollview(wcov[2], sub=[3, 3, 3], title="IU", cmap="coolwarm")
    hp.mollview(wcov[3], sub=[3, 3, 5], title="QQ", cmap="coolwarm")
    hp.mollview(wcov[4], sub=[3, 3, 6], title="QU", cmap="coolwarm")
    hp.mollview(wcov[5], sub=[3, 3, 9], title="UU", cmap="coolwarm")
    
    return 'Good'
    

# PLAY WITH SCALES

In [ ]:
# Parameters related to resolution
nside_inputmap = 64
nside_mapmaking = 64
nnz_manual = 3 #expects 3

res = [nside_inputmap,nside_mapmaking,nnz_manual]

In [ ]:
Whichmap = 'sim_map_fixed.fits' # 'sim_map.fits' to use a newly generated map, 'sim_map_fixed.fits' to use a premade map
SetHubble = 67 #Setting the cosmological parameter: Hubble Constant (km/s/Mpc)
SetUnit = 'muK' #of temperature
SetLmax = 1024 #Cf. Original input: 1024
InputType = 1 #0 for Gaussian CLS with arbitrary scale, 1 for CMB
InputNside = nside_inputmap
SetFWHM = np.radians(1) #can be set to 0 or np.radians(wanted value (orig. 1))

signalparams = [Whichmap,SetHubble,SetUnit,SetLmax,InputType,InputNside,SetFWHM]

In [ ]:
sr = 113  # sample rate of the detectors (per second)
scandur = '1 Day' #Scan duration,format: 'N Day(Days) N Hour(Hours) N Minute(Minutes) N Second(Seconds)'
scr = 120  # scan rate, degrees per second
sc_ac = 0 # scan acceleration
SetBaseline = 10

scan = [sr,scandur,scr,sc_ac,SetBaseline]
t_ = readtime(scandur)

In [ ]:
### Play with Scale. 

# Detector Noise Factos
NET = 12 #GB: 12, TOAST default: 1
FKnee = 0.1 #GB: 0.1, TOAST default: 0.05

# Gain of signal and atmospheric noise (through the detector)
Gain = 3e-5*(10**(6)) # 3e-5 times the factor e+6, because the atmospheric noise is given by K, not muK.
    # This gain was calibrated against POLARBEAR data (3e-5) # the result should be considered along with this factor
    # ratio when converting signal to voltage.

In [ ]:
import time
t00 = time.time()

datatuple = DefineOBS_data(MakeInputData(FocPlane(wh,FKnee,NET),wf,sr,t_,scr))
mapmachen(scan,res,signalparams,Gain,datatuple)

t99 = time.time()
dt = t99-t00
print('----------')
print('Total Mapmaking Duration: '+str(int(dt//60))+' minutes, '+str(int(round((dt-60*(dt//60)),0)))+' seconds')